In [1]:
# Import core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
# Display settings
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid', palette='muted')

# --- 2. LOAD DATA ---
# (For Kaggle Notebook use absolute path; for GitHub, assume file in /data/)

import kagglehub
# Download latest version
path = kagglehub.dataset_download("anthonygiorgio/ifsc-climbing-competition-results-1991-2019")
#data_path = 'kagglehub/datasets/anthonygiorgio/ifsc-climbing-competition-results-1991-2019/versions/2'
df = pd.read_excel(path)





Using Colab cache for faster access to the 'ifsc-climbing-competition-results-1991-2019' dataset.


IsADirectoryError: [Errno 21] Is a directory: '/kaggle/input/ifsc-climbing-competition-results-1991-2019'

In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("anthonygiorgio/ifsc-climbing-competition-results-1991-2019")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/anthonygiorgio/ifsc-climbing-competition-results-1991-2019/versions/2
